Central Park's coordinate system points are
40.77898°, -73.96925°
40.77898,-73.96925

Based on instructions found in
https://weather-gov.github.io/api/general-faqs

Retrieve the metadata for your location from https://api.weather.gov/points/{lat},{lon}.
For our example the URL will be https://api.weather.gov/points/40.77898,-73.96925

This retrieves the following
"gridId": "OKX",
"gridX": 33,
"gridY": 37,

Based on the retrieved information, the api url call can be formatted as follows
url = 'https://api.weather.gov/gridpoints/OKX/33,37/forecast'

In [1]:
# Central Park 14 day Weather Forecast
# 40.77898,-73.96925
import requests
import json
import pandas as pd

# set the API endpoint URL for the coordinates of the Central Park weather station
url = 'https://api.weather.gov/gridpoints/OKX/33,37/forecast'

# send the API request
response = requests.get(url)

# parse the JSON response
data = json.loads(response.text)

# create an empty list to hold the forecast data
forecast_data = []

# loop over each forecast period
for period in data['properties']['periods']:
    # create a dictionary for the forecast data
    forecast = {
        'timestamp': period['startTime'],
        'temperature': period['temperature'],
        'temperature_unit': period['temperatureUnit'],
        'wind_speed': period['windSpeed'],
        'wind_direction': period['windDirection'],
        'short_forecast': period['shortForecast'],
        'detailed_forecast': period['detailedForecast'],
        'precipitation_chance': period['probabilityOfPrecipitation']['value']
    }
    forecast_data.append(forecast)

forecast_df = pd.DataFrame(forecast_data)
forecast_df

,timestamp,temperature,temperature_unit,wind_speed,wind_direction,short_forecast,detailed_forecast,precipitation_chance
0,2023-03-04T23:00:00-05:00,36,F,6 mph,NW,Mostly Cloudy,"Mostly cloudy, with a low around 36. Northwest...",NaN
1,2023-03-05T06:00:00-05:00,52,F,6 to 12 mph,NW,Mostly Sunny,"Mostly sunny, with a high near 52. Northwest w...",NaN
2,2023-03-05T18:00:00-05:00,36,F,10 mph,NW,Mostly Clear,"Mostly clear, with a low around 36. Northwest ...",NaN
3,2023-03-06T06:00:00-05:00,54,F,10 mph,NW,Sunny,"Sunny, with a high near 54. Northwest wind aro...",NaN
4,2023-03-06T18:00:00-05:00,38,F,9 mph,NW,Rain And Snow Likely,"Rain and snow likely after 7pm. Mostly cloudy,...",60.0
5,2023-03-07T06:00:00-05:00,45,F,7 to 18 mph,NW,Rain And Snow Likely,Rain and snow likely before 1pm. Mostly cloudy...,60.0
6,2023-03-07T18:00:00-05:00,34,F,18 mph,NW,Partly Cloudy,"Partly cloudy, with a low around 34.",NaN
7,2023-03-08T06:00:00-05:00,47,F,18 mph,NW,Mostly Sunny,"Mostly sunny, with a high near 47.",NaN
8,2023-03-08T18:00:00-05:00,35,F,13 to 17 mph,NW,Partly Cloudy,"Partly cloudy, with a low around 35.",NaN
9,2023-03-09T06:00:00-05:00,48,F,14 mph,NW,Mostly Sunny,"Mostly sunny, with a high near 48.",NaN


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Set up the URL and headers
url = 'https://www.accuweather.com/en/us/central-park/10028/daily-weather-forecast/2627448'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

# Get the page content
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# Find the content-module
content_module = soup.find('div', class_='page-column-1 page-column')

# Create an empty dataframe
df = pd.DataFrame(columns=['date', 'high', 'low'])

# Find the daily-wrapper classes and loop through them
daily_wrappers = content_module.find_all('div', class_='daily-wrapper')
for daily in daily_wrappers:
    # Find the high and low temperatures
    high = daily.find('div', class_='high').text.strip()
    low = daily.find('div', class_='low').text.strip()
    
    # Find the date and format it
    date = daily.find('h2').text.strip()
    date = pd.to_datetime(date + ' ' + str(pd.Timestamp.now().year))
    
    # Add the data to the dataframe
    df = df.append({'date': date, 'high': high, 'low': low}, ignore_index=True)

# Print the dataframe
print(df)


In [18]:
# https://medium.com/@yaya.nurcahya/web-scraping-using-python-part-2-scraping-accuweather-8fb9860df1b
from bs4 import BeautifulSoup
from fake_headers import Headers
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager


def get_city_list_by_keyword(keyword):
    url = f'https://www.accuweather.com/en/search-locations?query={keyword}'

    options = Options()
    options.headless = True

    header = Headers(
        browser="chrome",  # Generate only Chrome UA
        os="mac",  # Generate any Mac platform
        headers=False,  # generate misc headers
    )
    custom_user_agent = header.generate()['User-Agent']
    options.add_argument(f"user-agent={custom_user_agent}")

    driver_path = ChromeDriverManager().install()
    driver = webdriver.Chrome(executable_path=driver_path, options=options)
    driver.get(url)

    soup = BeautifulSoup(driver.page_source, 'lxml')
    driver.close()

    try:
        tag_a_list = soup.find('div', attrs={'class': 'locations-list content-module'}).find_all('a', href=True)
    except AttributeError:
        print(f'\n░░░ No data found (london) ░░░')
        tag_a_list = None

    count_list = 0
    if tag_a_list is not None:
        for a in tag_a_list:
            count_list += 1
            href = a['href']
            city = a.getText(strip=True)
            print(f'{count_list}. {city} => https://www.accuweather.com{href}')


get_city_list_by_keyword('london')

C:\Users\cjong\AppData\Local\Temp\ipykernel_23172\4260065302.py:13: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True
C:\Users\cjong\AppData\Local\Temp\ipykernel_23172\4260065302.py:24: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path, options=options)


1. London, London, GB => https://www.accuweather.com/web-api/three-day-redirect?key=328328&target=
2. London, Ontario, CA => https://www.accuweather.com/web-api/three-day-redirect?key=55489&target=
3. Siguel, South Cotabato, PH => https://www.accuweather.com/web-api/three-day-redirect?key=3422743&target=
4. London, Kentucky, US => https://www.accuweather.com/web-api/three-day-redirect?key=333298&target=
5. London, Ohio, US(43140) => https://www.accuweather.com/web-api/three-day-redirect?key=335012&target=
6. London, Line Islands, KI => https://www.accuweather.com/web-api/three-day-redirect?key=1123888&target=
7. London, Arkansas, US => https://www.accuweather.com/web-api/three-day-redirect?key=2123173&target=
8. London, California, US => https://www.accuweather.com/web-api/three-day-redirect?key=2154402&target=
9. London, Texas, US(75684) => https://www.accuweather.com/web-api/three-day-redirect?key=2-2103712_1_al&target=
10. London, Altay, RU => https://www.accuweather.com/web-api/thr

In [20]:
from bs4 import BeautifulSoup
from fake_headers import Headers
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager


# Set up the URL and headers
url = 'https://www.accuweather.com/en/us/central-park/10028/daily-weather-forecast/2627448'
# headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

options = Options()
options.headless = True

header = Headers(
    browser="chrome",  # Generate only Chrome UA
    os="mac",  # Generate any Mac platform
    headers=False,  # generate misc headers
)
custom_user_agent = header.generate()['User-Agent']
options.add_argument(f"user-agent={custom_user_agent}")

driver_path = ChromeDriverManager().install()
driver = webdriver.Chrome(executable_path=driver_path, options=options)
driver.get(url)

print(f'test1 {driver.page_source}')
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.close()

# Find the content-module
content_module = soup.find('div', class_='header-placeholder has-alerts ')
print(f'test {content_module}')





# Find the content-module
content_module = soup.find('div', class_='page-column-1 page-column')
print(f'test {content_module}')

# Create an empty dataframe
df = pd.DataFrame(columns=['date', 'high', 'low'])

# Find the daily-wrapper classes and loop through them
daily_wrappers = content_module.find_all('div', class_='daily-wrapper')
for daily in daily_wrappers:
    # Find the high and low temperatures
    high = daily.find('div', class_='high').text.strip()
    low = daily.find('div', class_='low').text.strip()
    
    # Find the date and format it
    date = daily.find('h2').text.strip()
    date = pd.to_datetime(date + ' ' + str(pd.Timestamp.now().year))
    
    # Add the data to the dataframe
    df = df.append({'date': date, 'high': high, 'low': low}, ignore_index=True)

# Print the dataframe
df


C:\Users\cjong\AppData\Local\Temp\ipykernel_23172\3610839897.py:13: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True
C:\Users\cjong\AppData\Local\Temp\ipykernel_23172\3610839897.py:24: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path, options=options)


test1 <html lang="en-us" class="accuweather"><head>
	<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
	
	<meta charset="utf-8">
	<link rel="canonical" href="https://www.accuweather.com/en/us/central-park/10028/daily-weather-forecast/2627448">
	<title>Central Park, NY Daily Weather | AccuWeather</title>
	<meta name="Description" content="Know what's coming with AccuWeather's extended daily forecasts for Central Park, NY. Up to 90 days of daily highs, lows, and precipitation chances.">
	<meta name="viewport" content="width=device-width, initial-scale=1.0">
	<meta name="referrer" content="origin">
	
	

	<meta property="fb:profile_id" content="AccuWeather">
<meta property="fb:app_id" content="132437483467956">
<meta property="fb:pages" content="71781612888">
<meta property="og:type" content="website">
<meta property="og:title" content="Central Park, NY Daily Weather | AccuWeather">
<meta property="og:description" content="Know what's coming with AccuWeather's extended daily f

AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
# https://medium.com/@yaya.nurcahya/web-scraping-using-python-part-2-scraping-accuweather-8fb9860df1b
from bs4 import BeautifulSoup
from fake_headers import Headers
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager


def get_city_list_by_keyword(keyword):
    url = f'https://www.accuweather.com/en/search-locations?query={keyword}'

    options = Options()
    options.headless = True

    header = Headers(
        browser="chrome",  # Generate only Chrome UA
        os="mac",  # Generate any Mac platform
        headers=False,  # generate misc headers
    )
    custom_user_agent = header.generate()['User-Agent']
    options.add_argument(f"user-agent={custom_user_agent}")

    driver_path = ChromeDriverManager().install()
    driver = webdriver.Chrome(executable_path=driver_path, options=options)
    driver.get(url)

    soup = BeautifulSoup(driver.page_source, 'lxml')
    driver.close()

    try:
        tag_a_list = soup.find('div', attrs={'class': 'locations-list content-module'}).find_all('a', href=True)
    except AttributeError:
        print(f'\n░░░ No data found (london) ░░░')
        tag_a_list = None

    count_list = 0
    if tag_a_list is not None:
        for a in tag_a_list:
            count_list += 1
            href = a['href']
            city = a.getText(strip=True)
            print(f'{count_list}. {city} => https://www.accuweather.com{href}')


get_city_list_by_keyword('london')